In [1]:
import json
import os
import warnings

from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from pathlib import Path
import requests
import yaml

In [2]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 250)

In [3]:
# Current utc, yeat, month and day
# utcnow = datetime.utcnow().strftime("%Y%m%d_%H%M")
dateoftoday = datetime.utcnow().strftime("%Y%m%d")
year = datetime.utcnow().strftime("%Y")
month = datetime.utcnow().strftime("%m")
day = datetime.utcnow().strftime("%d")

### Nacka : 17853

In [4]:
nacka = [
    473483,
    473487,
    924072,
    473485,
    473486,
    902893,
    924002,

    923670,
    923806,
    909907,
    910084,
    924061,
    924062,
    909686,
    924053,
    473282,
    473484,
    909685,
    909942,
    909913,
    473290,
    909894,
    952513,
    953779,
    944683,
    946236,
    951329,
    949959,
    946237,
    944765,
    953780,
    953781,
    952381,
    944682,
    953360
]

In [5]:
    902894,

(902894,)

In [6]:
# Loop thruogh all the areas one by one
nacka_df_list = []

for area_code in nacka:
    print(area_code)

    # Start the process of scaring
    url = "https://www.hemnet.se/bostader"
    params= {
        #"housing_form_groups":"apartments",
        "location_ids":area_code,
        #"item_types":"bostadsratt",
        "rooms_min":0,
        "living_area_min":0,
        "new_construction":"include"
    }
    payload={}
    headers = {"User-Agent": "Mozilla/5.0"
    }

    response = requests.request(
        "GET",
        url,
        headers=headers,
        data=payload,
        params=params
    )

    soup = BeautifulSoup(response.content, "html.parser")
    map_results=soup.find(id="results-map")
    initial_data=map_results.attrs["data-initial-data"]
    json_data=json.loads(initial_data)

    url = "https://www.hemnet.se/bostader/search/"+json_data["search_key"]

    params= {}
    payload={}

    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.request(
        "GET",
        url,
        headers=headers,
        data=payload,
        params=params
    )

    r_json = response.json()
    r_properties = r_json["properties"]

    # Store all the listings in that area as a dataframe
    df = pd.json_normalize(
        r_properties,
        max_level=1
    )

    print(f"{df.shape}")

    # Enrich the df with info about internal area naming and date
    df["area"] = "nacka"
    df["date"] = dateoftoday 
    
    nacka_df_list.append(df)

473483
(211, 44)
473487
(44, 44)
924072
(52, 44)
473485
(132, 44)
473486
(41, 44)
902893
(2, 40)
924002
(18, 44)
923670
(39, 44)
923806
(36, 41)
909907
(5, 41)
910084
(8, 43)
924061
(9, 43)
924062
(14, 40)
909686
(23, 44)
924053
(1, 43)
473282
(21, 44)
473484
(35, 40)
909685
(163, 44)
909942
(73, 44)
909913
(5, 41)
473290
(65, 44)
909894
(6, 40)
952513
(8, 40)
953779
(21, 41)
944683
(21, 43)
946236
(28, 44)
951329
(8, 43)
949959
(5, 41)
946237
(5, 43)
944765
(20, 44)
953780
(1, 40)
953781
(2, 40)
952381
(1, 40)
944682
(2, 40)
953360
(5, 41)


In [7]:
nacka_complete_df = pd.concat(nacka_df_list)
nacka_complete_df.drop_duplicates(subset="id", inplace=True)
nacka_complete_df.shape

(526, 46)

In [8]:
nacka_complete_df.reset_index(inplace=True)
nacka_complete_df.drop("index", axis=1, inplace=True)
nacka_complete_df

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,upcoming,price,price_per_area,has_price_change,rooms,fee,age_string,age,deactivated_before_open_house,land_area,small_image_url,medium_image_url,url,labels,state_labels,visible_project_units,show_visible_project_units,user_saved,save_button_label,broker_logo_url,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,broker_name,broker_phone,broker_email,price_change,area,date
0,20042922,"[59.31741242397093, 18.161689411102365]","Augustendalsvägen 23, 3 tr",Nacka kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,8 995 000 kr,79 602 kr/m²,False,4 rum,5 905 kr/mån,<span>1</span> dagar,1,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-4rum-nac...,"[{'category': 'feature', 'identifier': 'patio'...",[],0,False,False,Spara,https://bilder.hemnet.se/images/broker_logo_1/...,BOSTHLM Nacka,"Nacka - Nacka Strand, Nacka kommun",False,False,"{'living_area': '113 m²', 'supplemental_area':...",20042922,common_listing,not_selected,active,for_sale,NaN,NaN,NaN,NaN,nacka,20230527
1,20039836,"[59.3171078, 18.1728267]",Trillans väg 4,Nacka kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,2 995 000 kr,47 165 kr/m²,False,2 rum,4 536 kr/mån,<span>1</span> dagar,1,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-2rum-jar...,"[{'category': 'feature', 'identifier': 'balcon...",[],0,False,False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,SkandiaMäklarna Nacka,"Jarlaberg, Nacka kommun",False,False,"{'living_area': '63,5 m²', 'supplemental_area'...",20039836,common_listing,not_selected,active,for_sale,NaN,NaN,NaN,NaN,nacka,20230527
2,19736395,"[59.3179373553, 18.1629823852]",Augustendalsvägen 29,Nacka kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,2 995 000 kr,61 122 kr/m²,False,2 rum,2 861 kr/mån,<span>1</span> dagar,1,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-2rum-nac...,"[{'category': 'product', 'identifier': 'premiu...","[{'category': 'state', 'identifier': 'highligh...",0,False,False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Nacka,"Nacka Strand, Nacka kommun",False,True,"{'living_area': '49 m²', 'supplemental_area': ...",19736395,common_listing,premium,active,for_sale,NaN,NaN,NaN,NaN,nacka,20230527
3,20036616,"[59.3149834, 18.1246548]",Finnboda Varvsväg 12A,Nacka kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,3 495 000 kr,74 362 kr/m²,False,2 rum,2 621 kr/mån,<span>1</span> dagar,1,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-2rum-nac...,"[{'category': 'product', 'identifier': 'premiu...","[{'category': 'state', 'identifier': 'highligh...",0,False,False,Spara,https://bilder.hemnet.se/images/broker_logo_1/...,Widerlöv Nacka/Värmdö,"NACKA, Nacka kommun",False,True,"{'living_area': '47 m²', 'supplemental_area': ...",20036616,common_listing,premium,active,for_sale,NaN,NaN,NaN,NaN,nacka,20230527
4,20036995,"[59.31588674813135, 18.141928732723855]","Kvarnholmsvägen 100, Högst upp",Nacka kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,3 495 000 kr,87 375 kr/m²,False,2 rum,4 035 kr/mån,<span>1</span> dagar,1,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-2rum-nac...,"[{'category': 'product', 'identifier': 'premiu...",[]

In [9]:
nacka_complete_df.to_parquet("../data/stockholms_lan/2023/05/27/nacka.parquet.gzip", compression="gzip")